# Ghi dữ liệu ra Kafka

Ở phần này, ta sẽ miêu tả việc hỗ trợ cho ghi Truy vấn trực tiếp theo dòng và Tập hợp truy vấn ra Apache Kafka. **Nhớ rằng Apache Kafka chỉ đảm bảo việc ghi ít nhất một lần**. Hệ quả là, khi viết, dù là Truy vấn trực tiếp (Streaming Queries) hay Tập hợp Truy vấn (Batch Queries) đến Kafka, một số bản ghi có thể bị trùng lặp. Điều này có thể xảy ra, ví dụ, khi Kafka muốn gửi lại một tin nhắn mà không được xác nhận bởi Broker, mặc dù Broker đã nhận và đã ghi lại tin nhắn đó. Cấu trúc truyền trực tiếp không thể ngăn được sự trùng lặp này diễn ra do cách Kafka viết. Tuy nhiên, nếu quá trình ghi lại truy vấn diễn ra thành công, ta có thể giả định là kết quả của truy vấn được viết ít nhất một lần. Một giải pháp khả thi để loại bỏ việc trùng lặp này là khi đọc dữ liệu có thể sử dụng một khoá chính để tránh trùng lặp khi đọc.

Dataframe viết vào Kafka sẽ có cấu trúc các cột như sau

|                         Column                    |                                  Type                                   |
| ------------------------------------------------- | ----------------------------------------------------------------------- |
| Khoá (key) (tuỳ chọn)                             | string hoặc binary                                                      |
| Giá trị (value) (bắt buộc)                        | string hoặc binary                                                      |
| Phần đầu (Headers) (tuỳ chọn)                     | mảng                                                                    |
| Chủ đề (Topic) (tuỳ chọn)                         | string                                                                  |
| Phân vùng (Partition) (tuỳ chọn)                  | int                                                                     |

**Lưu ý: Với trường topic, nếu trong cấu hình chưa được chỉ ra thì trường này phải bắt buộc được chỉ rõ ở trong dataframe**

Cột giá trị là cột duy nhất bắt buộc phải có. Nếu cột khoá không được chỉ ra thì khoá sẽ được gán giá trị "null" một cách tự động (xem cách Kafka viết (Kafka semantics) để hiểu hơn cách null được xử lý). Nếu cột chủ đề tồn tại thì giá trị của nó được sử dụng như chủ đề khi viết vào một dòng (row) đến Kafka, trừ khi cấu hình "topic" đã được thiết lập. Tức là cấu hình "topic" sẽ ghi đè lên cột topic. Nếu cột "partition" khôn được chỉ ra (hoặc có nhưng giá trị là null) thì phân vùng sẽ được tính toán và xác định bởi Kafka producer. Một phân vùng Kafka có thể được chỉ định bởi Spark trong cài đặt tuỳ chọn của kafka.partitioner.class. Nếu không được nói đến, phân vùng Kafka mặc định sẽ được sử dụng.

Các tuỳ chọn sau phải được sử dụng cho Kafka sink cho cả truy vấn tập hợp và trực tiếp theo dòng:
    
| Tuỳ chọn | Giá trị | Ý nghĩa |
| :---------------------- | :----------------------------- | :--------------------------------------------------------------- |
| kafka.bootstrap.servers | Một danh sách "host:port" phân cách bởi dấu phẩy | Cấu hình server Kafka "bootstrap.servers"      |

Các trường cấu hình sau là tuỳ chọn:
    
| Tuỳ chọn | Giá trị | Mặc định | Kiểu truy vấn | Ý nghĩa |
| :------- | :------ | :------- | :------------------ | :-------------------------------------------------------------------- |
| Chủ đề (topic) | string | none | Trực tiếp và tập hợp | Thiết lập chủ đề mà tất cả các dòng sẽ được ghi vào trong Kafka. Trường này khi được thiết lập sẽ ghi đè lên tất cả các cột topic (nếu có) trong dữ liệu. |
| includeHeaders (bao gồm phần đầu) | boolean | false |  Trực tiếp và tập hợp | Khi muốn bao gồm phần đầu trong các dòng dữ liệu |

In [1]:
!pip install kafka-python

để có thể sử dụng kafka thì chúng ta cần khai báo một số dòng lệnh như phía dưới

In [2]:
import os 
packages = "org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.1"

os.environ["PYSPARK_SUBMIT_ARGS"] = (
    "--packages {0} pyspark-shell".format(packages)
)

from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import to_json, struct, lit
sc = SparkContext('local')
spark = SparkSession(sc)

KAFKA_BROKER = "kafka:9092"
KAFKA_TOPIC = "default_topic"


tiến hành đọc file csv và in ra thử data

In [3]:
df = (spark.read.format("com.databricks.spark.csv")
        .option("header", "true")
        .option("inferSchema","true")
        .load("../data/census_1000.csv"))
        
df_list = df.collect()
df.show(3, False)

+---+---+-----------------+----------+-------------+-------------------+------------------+--------------+---------+------+------------+------------+--------------+------+
|_c0|age|workclass        |education |education-num|marital-status     |occupation        |relationship  |ethnicity|gender|capital-gain|capital-loss|hours-per-week|loan  |
+---+---+-----------------+----------+-------------+-------------------+------------------+--------------+---------+------+------------+------------+--------------+------+
|0  |39 | State-gov       | Bachelors|13           | Never-married     | Adm-clerical     | Not-in-family| White   | Male |2174        |0           |40            | <=50K|
|1  |50 | Self-emp-not-inc| Bachelors|13           | Married-civ-spouse| Exec-managerial  | Husband      | White   | Male |0           |0           |13            | <=50K|
|2  |38 | Private         | HS-grad  |9            | Divorced          | Handlers-cleaners| Not-in-family| White   | Male |0           |0   

## Ghi đầu ra của truy vấn vào kafka
có 2 cách để có thể ghi dữ liệu từ dataframe đến kafka
1. sử dụng KafkaProducer
2. sử dụng write của spark

ví dụ dưới mô tả cách sử dụng KafkaProducer để ghi dữ liệu từ dataframe lên kafka

In [ ]:
import time
import json
import random
import logging

from kafka import KafkaProducer
from kafka.errors import KafkaError

producer = KafkaProducer(bootstrap_servers=[KAFKA_BROKER])
index = 0

while True:
    
    row_dict = df_list[index].asDict()
    
    future = producer.send(
        topic=KAFKA_TOPIC, 
        key=str(row_dict["_c0"]).encode("utf-8"),
        value=json.dumps(row_dict).encode("utf-8"))
    
    try:
        record_metadata = future.get(timeout=10)
    except KafkaError:
        # Decide what to do if produce request failed...
        logging.exception("Error")
        pass
    
    producer.flush()
    
    index += 1
    time.sleep(random.uniform(0.1,3.0))

ví dụ tiếp theo mô tả cách ghi dữ liệu lên kafka sử dung hàm write của spark

do kafka chỉ hỗ trợ gửi dữ liệu định dạng key:value nên ta cần phải chỉnh sửa dữ liệu trước khi gửi
ở đây ta tạo 1 cột mới có tên value với dữ liệu là dữ liệu của từng hàng trong dataframe với định dạng json

In [4]:
df = df.withColumn('value' ,to_json(struct([df[x] for x in df.columns])))
df.selectExpr("CAST(_c0 AS STRING) as key", "CAST(value AS STRING)").show(3, False)

+---+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|key|value                                                                                                                                                                                                                                                                                                      |
+---+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|0  |{"_c0":0,"age":39,"workclass":" State-gov","education":" Bachelors","educatio

để có thể gửi dữ liệu từ dataframe tới kafka ta cần ít nhất 2 cột, 1 cột key và 1 cột value. 
1. sử dụng format("kafka") để dịnh nghĩa việc gửi dữ liệu đến kafka
2. sử dụng option("kafka.bootstrap.servers", KAFKA_BROKER) để định nghĩa broker kafka đích, có thể định nghĩa nhiều broker dích ngăn cách nhau bới dấu ',' ví dụ "host1:port1,host2:port2"
3. sử dụng option("topic", KAFKA_TOPIC) để định nghĩa topic đích trong broker dích

ở đây ta chọn cột \_c0 là cột key và cột value chưa toàn bộ dữ liệu định dạng json ta vừa tạo ở trên

In [5]:
df.selectExpr("CAST(_c0 AS STRING) as key", "CAST(value AS STRING)")\
  .write\
  .format("kafka")\
  .option("kafka.bootstrap.servers", KAFKA_BROKER)\
  .option("topic", KAFKA_TOPIC)\
  .save()

ngoài cách định nghĩa 2 cột 1 cột key, 1 cột value thì ta cũng có thể sử dụng 1 cột trong dataframe để định nghĩa topic dích thay cho việc phải khai báo option topic gửi đến

dưới đây ta tạo 1 cột mới có tên là topic với toàn bộ dữ liệu là tên topic được định nghĩa trong biến KAFKA_TOPIC

In [6]:
df = df.withColumn('topic' ,lit(KAFKA_TOPIC))
df.selectExpr('topic', "CAST(_c0 AS STRING) as key", "CAST(value AS STRING)").show(3, False)

+-------------+---+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|topic        |key|value                                                                                                                                                                                                                                                                                                      |
+-------------+---+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|default_topic|0  |{"_c0":0,"age":39,"wo

ví dụ dưới sử dụng cột topic vừa tạo để định nghĩa topic gửi đến thay vì sử dụng option như ví dụ phía trên

In [7]:
df.selectExpr('topic',"CAST(_c0 AS STRING) as key", "CAST(value AS STRING)")\
  .write\
  .format("kafka")\
  .option("kafka.bootstrap.servers", KAFKA_BROKER)\
  .save()

## Tạo kafka sink để truyền stream dữ liệu
ngoài việc truyền dữ liệu từ truy vấn thì ta cũng có thể stream dữ liệu từ streaming dataframe lên kafka

trước khi có thể stream dữ liệu thì ta cần có stream dataframe 
việc tạo stream dataframe tương tự như trong **Chapter 21: Structured Streaming Basics**


In [8]:
static = spark.read.json("../data/activity-data/")
dataSchema = static.schema

đọc dữ liệu từ folder và lấy schema để có thể tạo stream dataframe

In [9]:
static.printSchema()

root
 |-- Arrival_Time: long (nullable = true)
 |-- Creation_Time: long (nullable = true)
 |-- Device: string (nullable = true)
 |-- Index: long (nullable = true)
 |-- Model: string (nullable = true)
 |-- User: string (nullable = true)
 |-- gt: string (nullable = true)
 |-- x: double (nullable = true)
 |-- y: double (nullable = true)
 |-- z: double (nullable = true)



In [10]:
static.show(4)

+-------------+-------------------+--------+-----+------+----+-----+------------+------------+------------+
| Arrival_Time|      Creation_Time|  Device|Index| Model|User|   gt|           x|           y|           z|
+-------------+-------------------+--------+-----+------+----+-----+------------+------------+------------+
|1424686735090|1424686733090638193|nexus4_1|   18|nexus4|   g|stand| 3.356934E-4|-5.645752E-4|-0.018814087|
|1424686735292|1424688581345918092|nexus4_2|   66|nexus4|   g|stand|-0.005722046| 0.029083252| 0.005569458|
|1424686735500|1424686733498505625|nexus4_1|   99|nexus4|   g|stand|   0.0078125|-0.017654419| 0.010025024|
|1424686735691|1424688581745026978|nexus4_2|  145|nexus4|   g|stand|-3.814697E-4|   0.0184021|-0.013656616|
+-------------+-------------------+--------+-----+------+----+-----+------------+------------+------------+
only showing top 4 rows



tạo stream dataframe với schema đọc được, tối đa 1 file mốĩ lần trigger từ folder

In [11]:
streaming = spark.readStream.schema(dataSchema).option("maxFilesPerTrigger", 1)\
.json("../data/activity-data")

cũng tương tự như việc gửi dữ liệu query thì việc gửi dữ liệu stream cũng cần tối thiểu 2 cột key và value
1. sử dụng writeStream để ghi stream dataframe lên kafka
2. sử dụng option("kafka.bootstrap.servers", KAFKA_BROKER) để định nghĩa broker đích, có thể định nghĩa nhiều broker dích ngăn cách nhau bới dấu ',' ví dụ "host1:port1,host2:port2"
3. sử dụng option("topic", KAFKA_TOPIC) để định nghĩa topic đích hoặc cos thể thay thế bằng việc định nghĩa cột topic trong dataframe
4. sử dụng option("checkpointLocation", "checkpoint") để lưu checkpoint quá trình stream data

ví dụ dưới tạo 1 cột mới có tên là value có dữ liệu là dữ liệu của từng hàng với định dạng json sau đó stream lên kafka

In [12]:
streaming.withColumn('value' ,to_json(struct([streaming[x] for x in streaming.columns])))\
  .selectExpr("CAST(Arrival_Time AS STRING) as key", "CAST(value AS STRING)") \
  .writeStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", KAFKA_BROKER) \
  .option("topic", KAFKA_TOPIC) \
  .option("checkpointLocation", "checkpoint")\
  .start()

ví dụ dưới tạo 1 cột topic và điền toàn bộ giá trị là là trị được định nghĩa trong biến KAFKA_TOPIC sau đó stream lên kafka sử dụng 3 cột topic, key, value thay cho việc sử dụng option để định nghĩa topic

In [13]:
streaming.withColumn('topic' ,lit(KAFKA_TOPIC))\
  .withColumn('value' ,to_json(struct([streaming[x] for x in streaming.columns])))\
  .selectExpr('topic', "CAST(Arrival_Time AS STRING) as key", "CAST(value AS STRING)") \
  .writeStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", KAFKA_BROKER) \
  .option("checkpointLocation", "checkpoint")\
  .start()

## Producer caching
do thực thể kafka producer được thiết kế để an toàn cho luồng, spark khởi tạo thực thể kafka producer và đồng sử dụng giữa các tác vụ với cùng caching key

caching key được xây dựng từ thông tin sau
- cấu hình kafka producer

nó bao gồm cấu hình để ủy quyền, spark sẽ tự động bao gồm khi token ủy quyền được sử dụng. ngay cả khi chúng ta lấy ủy quyền vào account, chúng ta có thể mong đợi rằng cùng thực thể kafka producer được sử dụng với cùng cấu hình kafka producer. nó sẽ dùng thực thể kafka producer mới nếu token ủy quyền được làm mới. thực thể kafka producer với token ủy quyền cũ sẽ bị loại bỏ theo chính sách bộ nhớ đệm. các thuộc tính sau có sẵn để cấu hình nhóm producer:

| tên thuộc tính | Mặc định | ý nghĩa | kể từ phiên bản|
| :--- | :--- |:--- | :--- |
| spark.kafka.producer.cache.timeout | 10 phút | khoảng thời gian nhỏ nhất mà 1 Producer không hoạt động trong pool trước khi bị loại bỏ | 2.2.1|
|spark.kafka.producer.cache.evictorThreadRunInterval| 1 phút| khoảng thời gian dừng giữa các lần chạy của luồng evictor cho nhóm producer. Khi được được thiết lập là non-positive, không có luồng evictor nào được chạy.| 3.0.1|
